# GEOG 497 - Spring 2021 - Cryosphere & Climate Systems
## A6: Remote sensing of ice sheet surface melt

## Part 1: Plotting AWS-derived melt data and co-located ASCAT backscatter

Input data: 
1) Enhanced spatial resolution (4.45-km) near-daily ASCAT data. Here we're using "all-pass" (msfa) data for the Antarctic region. The data have been converted from the natively-provided SIR format to yearly netcdfs. 
* Source: https://www.scp.byu.edu/data/Ascat/SIR/Ascat_sir.html
* Note if not running on Penn State's Roar system: you will need to download these data independently and convert to netcdfs. Future revisions of this assignment could instead provide time series of ASCAT data at the AWS sites. 

2) Estimates of in-situ surface meltwater production from Antarctic AWS sites.
* Source: https://doi.pangaea.de/10.1594/PANGAEA.910473
* In the code block below, data from 10 AWS sites will be downloaded from the PANGAEA data repository.
        
For this assignment, base code exist below that you can edit to plot ASCAT data alongside in situ melt rate data for any of the AWS sites (except AWS4, that pre-dates the ASCAT era). 

### Run the following two code blocks first.
The first downloads all of the AWS data. This only needs to be done once!

The next loads python packages and sets some plotting-related variables. 

In [ ]:
#Get all of the AWS data from the source

import urllib.request

urls = ['https://doi.pangaea.de/10.1594/PANGAEA.910467?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910470?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910471?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910477?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910479?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910480?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910483?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910452?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910465?format=textfile',
        'https://doi.pangaea.de/10.1594/PANGAEA.910466?format=textfile']

filenames = ['IMAU_aws11_high-res_meteo.tab',
             'IMAU_aws14_high-res_meteo.tab',
             'IMAU_aws15_high-res_meteo.tab',
             'IMAU_aws16_high-res_meteo.tab',
             'IMAU_aws17_high-res_meteo.tab',
             'IMAU_aws18_high-res_meteo.tab',
             'IMAU_aws19_high-res_meteo.tab',
             'IMAU_aws4_high-res_meteo.tab',
             'IMAU_aws5_high-res_meteo.tab',
             'IMAU_aws6_high-res_meteo.tab']

# loop through and download AWS data to Data folder
for i in range(len(urls)):
    print('Downloading: ' + filenames[i])
    urllib.request.urlretrieve(urls[i],'Data/' + filenames[i])

In [ ]:
# Import python packages

# for file searching
import glob 

# for data reading/analysis
import xarray as xr
import pandas as pd

#  for geographic projections
from pyproj import Transformer

# for plotting
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

# seaborn adds some extra visual appeal to our plots
import seaborn as sns

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# set some universal plot settings here
plt.rcParams["figure.dpi"] = 200
plt.rcParams['axes.xmargin'] = 0.05
sns.set_style('darkgrid')
sns.set_context("notebook", font_scale=0.75)
%config InlineBackend.figure_format = 'retina' # make high res plots for retina 5k displays

# lastly, this specifies where the AWS and ASCAT data are located
awsFolder = './Data/'
ascatFolder = '/gpfs/group/ljt5282/default/ASCAT_data/netcdfs/annual/'

### Preliminary step 1: read all the ASCAT data files
This reads all of the continent-wide, daily temporal resolution ASCAT netcdf files using `xarray`. Afterward, we will subset these data to only the grid cell nearest to the AWS location of interest.

In [ ]:
# read all yearly netcdfs using xarray and create new xarray DataSet variable
ascat_ds = xr.open_mfdataset(ascatFolder + 'msfa*.nc')

### Main code: Plot ASCAT and AWS melt data at one location
Code here is all set up to work and should be used as a reference for making plots at other AWS sites.

In the block directly below, you'll want to specify what AWS datafile we'll want to read.
This then loads it using `pandas` and resamples the hourly data to daily means.

In [ ]:
# Use pandas to read SEB melt data for the AWS site
aws_datafile = 'IMAU_aws14_high-res_meteo.tab'
aws_folder_plus_file = awsFolder + aws_datafile
aws_df = pd.read_csv(aws_folder_plus_file,
                            sep='\t', 
                            skiprows=40,
                            parse_dates=['Date/Time'],
                            index_col=['Date/Time'])

# Resample hourly data to daily means and set as new pandas DataFrame variable
aws_df_daily = aws_df.resample('D').mean()

In [ ]:
# Can use Pandas here to find when the AWS record starts and ends, and we'll use this to subset our ASCAT data in time
# str() here turns the timestamp into a string and allows us to combine it with our text below
# Note: this is only for our own information right now, but you can see below where we subset the ASCAT data where this is useful.

print('First observation at AWS: ' + str(aws_df_daily.index.min()))
print('Last observation at AWS: ' + str(aws_df_daily.index.max()))

### Define the latitude and longitude of where the AWS is located. 
This info can be found in the AWS data file's header:

In [ ]:
# here we're using a shell command 'head' to read the first few lines of the file
# then we're piping it through the 'grep' command to search for text we're interested in.
# this is also a useful point to verify that we're working with the AWS number we indend to work with

!head $aws_folder_plus_file | grep LATITUDE

In [ ]:
# Using that info from the header, specify here where this AWS is located (latitude and longitude)
aws_lat = 
aws_lon = 

### Use pyproj to convert lat/lon coordinates in the projected x/y coordinates of the ASCAT data:

In [ ]:
# from geographic lat/lon to EPSG:3976 (WGS 84 / NSIDC Sea Ice Polar Stereographic South)
transformer = Transformer.from_crs("epsg:4326", "epsg:3976")
aws_x,aws_y = transformer.transform(aws_lat, aws_lon)

### Subset the ASCAT data to the location and timespan of the AWS data

In [ ]:
# Now, subset the ascat data to only the point (i.e., the grid cell nearest to the AWS site) and  only when it overlaps with the AWS data
ascat_ds_ts_aws = ascat_ds['sigma0'].sel(x=aws_x, method="nearest") \
                                    .sel(y=aws_y, method="nearest") \
                                    .sel(time=slice(aws_df_daily.index.min(), aws_df_daily.index.max()))

### Plot the AWS-derived melt data and ASCAT data over that site

In [ ]:
# Now, plot the AWS and ASCAT data

# create a figure
fig, ax = plt.subplots(figsize=(15, 7.5))

# plot ascat data on first vetical axis
line1 = ax.plot(ascat_ds_ts_aws.time, ascat_ds_ts_aws,
                 label='ASCAT sigma0')

# make second vertical axis; plot aws data on it
ax2 = ax.twinx()
line2 = ax2.plot(aws_df_daily['Melt rate [mm w.e.] (surface melt, within dt)'], 
                label='AWS-derived melt rate', 
                color='indianred')

# Set some display properties
ax.set(ylabel='ASCAT backscatter at AWS site [dB]',
       xlabel='Date',
       title='AWS14')

ax2.set(ylabel='Daily mean melt rate [mm w.e.]')
ax2.yaxis.grid(False)

ax2.set_ylim((-0.5, 0.5))
ax.set_ylim((-30, 30))

# add a legend now that we have 2 lines
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc='upper left')

plt.show()